In [1]:
!pip install pyspark
!apt update
!apt install openjdk-8-jdk-headless -qq
import os
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark import SparkContext
from pyspark.sql import SQLContext
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.3-py2.py3-none-any.whl size=317840625 sha256=002dcca8090dc35f4c34604b450018af0d8f4ef953bd92097f954948fdd9392f
  Stored in directory: /root/.cache/pip/wheels/1b/3a/92/28b93e2fbfdbb07509ca4d6f50c5e407f48dce4ddbda69a4ab
Successfully built pyspark
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Ign:4 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:7 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:9 http

In [2]:
spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext
sqlContext = SQLContext(sc)
df = sqlContext.read.csv('transactions_train.csv', header=True, inferSchema=True)
transactions_rdd = df.rdd

/usr/local/lib/python3.10/dist-packages/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [145]:
from math import floor
def redondeo(numero, decimal):
  ajuste = 10**decimal
  return floor(numero * ajuste) / ajuste

In [3]:
transactions_rdd.first()

Row(_c0=0, Id=2987000, target=0, timestamp=86400, ammount=68.5, product_category='W', feat_card_1=13926, feat_card_2=None, feat_card_3=150.0, feat_card_4='discover', feat_card_5=142.0, feat_card_6='credit', addr1=315.0, addr2=87.0, dist1=19.0, dist2=None, payer_email_domain=None, receiver_email_domain=None, A1=1.0, A2=1.0, A3=0.0, A4=0.0, A5=0.0, A6=1.0, A7=0.0, A8=0.0, A9=1.0, A10=0.0, A11=2.0, A12=0.0, A13=1.0, A14=1.0, B1=14.0, B2=None, B3=13.0, B4=None, B5=None, B6=None, B7=None, B8=None, B9=None, B10=13.0, B11=13.0, B12=None, B13=None, B14=None, B15=0.0, C1='T', C2='T', C3='T', C4='M2', C5='F', C6='T', C7=None, C8=None, C9=None, D1=1.0, D2=1.0, D3=1.0, D4=1.0, D5=1.0, D6=1.0, D7=1.0, D8=1.0, D9=1.0, D10=0.0, D11=0.0, D12=1.0, D13=1.0, D14=1.0, D15=0.0, D16=0.0, D17=0.0, D18=0.0, D19=1.0, D20=1.0, D21=0.0, D22=0.0, D23=1.0, D24=1.0, D25=1.0, D26=1.0, D27=0.0, D28=0.0, D29=0.0, D30=0.0, D31=0.0, D32=0.0, D33=0.0, D34=0.0, D35=None, D36=None, D37=None, D38=None, D39=None, D40=None, D

Porcentaje de fraude

In [ ]:
transactions_rdd.map(lambda x: x.target).distinct().collect()

[0, 1]

In [146]:
redondeo(transactions_rdd.filter(lambda x: x.target == 1).count() * 100 / transactions_rdd.count(), 2)

3.5

Cantidad de monto respecto de los productos

In [80]:
lista_productos = transactions_rdd.map(lambda x: ((x.product_category, x.target), x.ammount)).reduceByKey(lambda a,b: a + b).cache()

In [152]:
sorted(lista_productos.map(lambda x: (x[0], redondeo(x[1], 1))).collect())

[(('C', 0), 2227829.5),
 (('C', 1), 330453.3),
 (('H', 0), 1986435.0),
 (('H', 1), 222872.0),
 (('R', 0), 5424125.0),
 (('R', 1), 293550.0),
 (('S', 0), 427847.1),
 (('S', 1), 37121.0),
 (('W', 0), 54909721.5),
 (('W', 1), 1730240.0)]

Porcentaje de monto respecto de los productos

In [87]:
monto_productos = lista_productos.map(lambda x: (1, x[1])).reduceByKey(lambda a,b: a + b).collect()[0][1]

In [154]:
def porcentaje(lista, total):
  promedios = []
  for producto in lista:
    promedios.append((producto[0], producto[1] * 100 / total))
  return promedios

sc.parallelize(porcentaje(sorted(lista_productos.collect()), monto_productos)).map(lambda x: (x[0], redondeo(x[1], 2))).collect()

[(('C', 0), 3.29),
 (('C', 1), 0.48),
 (('H', 0), 2.93),
 (('H', 1), 0.32),
 (('R', 0), 8.02),
 (('R', 1), 0.43),
 (('S', 0), 0.63),
 (('S', 1), 0.05),
 (('W', 0), 81.23),
 (('W', 1), 2.55)]

Cantidad de monto fraudulento respecto de los productos

In [155]:
fraude = lista_productos.filter(lambda x: x[0][1] == 1).map(lambda x: (x[0][0], redondeo(x[1], 2)))
fraude.takeOrdered(5, lambda x: -x[1])

[('W', 1730240.03),
 ('C', 330453.3),
 ('R', 293550.0),
 ('H', 222872.0),
 ('S', 37121.0)]

Cantidad de monto legal respecto de los productos

In [156]:
legal = lista_productos.filter(lambda x: x[0][1] == 0).map(lambda x: (x[0][0], redondeo(x[1], 2)))
legal.takeOrdered(5, lambda x: -x[1])

[('W', 54909721.53),
 ('R', 5424125.0),
 ('C', 2227829.57),
 ('H', 1986435.0),
 ('S', 427847.1)]

Porcentaje de monto fraudulento respecto de los productos

In [157]:
sc.parallelize(porcentaje(sorted(fraude.collect()), monto_productos)).map(lambda x: (x[0], redondeo(x[1], 2))).takeOrdered(5, lambda x: -x[1])

[('W', 2.55), ('C', 0.48), ('R', 0.43), ('H', 0.32), ('S', 0.05)]

Porcentaje de monto legal respecto de los productos

In [158]:
sc.parallelize(porcentaje(sorted(legal.collect()), monto_productos)).map(lambda x: (x[0], redondeo(x[1], 2))).takeOrdered(5, lambda x: -x[1])

[('W', 81.23), ('R', 8.02), ('C', 3.29), ('H', 2.93), ('S', 0.63)]

Fraude segun el tiempo, analizando cantidad y monto

In [161]:
fraude = transactions_rdd.filter(lambda x: x.target == 1).cache()

In [206]:
def segundos(numero):
  return numero - 86400

def analisis(numero, divisor):
  flotante = (numero - 86400) / divisor
  entero = (numero - 86400) // divisor
  if flotante > entero:
    return entero + 1
  else:
    return entero

def minutos(numero):
  return analisis(numero, 60)

def horas(numero):
  return analisis(numero, 3600)

def dias(numero):
  return analisis(numero, 86400)

def semanas(numero):
  return analisis(numero, 604800)

def meses(numero):
  return analisis(numero, 2419200)

def porcentaje_tupla(tupla, total):
  return (tupla[0], redondeo(tupla[1], 2), redondeo(tupla[1] * 100 / total, 2))

def porcentaje_lista(lista, total):
  nueva_lista = []
  for elemento in lista:
    nueva_lista.append((elemento[0], redondeo(elemento[1], 2), redondeo(elemento[1] * 100 / total, 4)))
  return nueva_lista

Cantidad respecto del tiempo

Cantidad total respecto de segundos

In [163]:
cantidad_total = fraude.count()
(fraude.map(lambda x: segundos(x.timestamp)).reduce(lambda a,b: a if a > b else b), cantidad_total, 100)

(13064705, 17580, 100)

Top ten cantidad respecto de minutos

In [207]:
porcentaje_lista(fraude.map(lambda x: (minutos(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), cantidad_total)

[(139695, 6.0, 0.0341),
 (136426, 6.0, 0.0341),
 (73412, 5.0, 0.0284),
 (148153, 5.0, 0.0284),
 (96611, 4.0, 0.0227),
 (60687, 4.0, 0.0227),
 (27508, 4.0, 0.0227),
 (176918, 4.0, 0.0227),
 (89024, 4.0, 0.0227),
 (140574, 4.0, 0.0227)]

Last ten cantidad respecto de minutos

In [208]:
porcentaje_lista(fraude.map(lambda x: (minutos(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), cantidad_total)

[(100, 1.0, 0.0056),
 (110, 1.0, 0.0056),
 (220, 1.0, 0.0056),
 (490, 1.0, 0.0056),
 (645, 1.0, 0.0056),
 (665, 1.0, 0.0056),
 (835, 1.0, 0.0056),
 (865, 1.0, 0.0056),
 (925, 1.0, 0.0056),
 (1000, 1.0, 0.0056)]

Top ten cantidad respecto de horas

In [209]:
porcentaje_lista(fraude.map(lambda x: (horas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), cantidad_total)

[(1529, 42.0, 0.2389),
 (1344, 36.0, 0.2047),
 (1530, 33.0, 0.1877),
 (2790, 32.0, 0.182),
 (2781, 32.0, 0.182),
 (2949, 31.0, 0.1763),
 (1026, 27.0, 0.1535),
 (1484, 26.0, 0.1478),
 (430, 25.0, 0.1422),
 (1531, 25.0, 0.1422)]

Last ten cantidad respecto de horas

In [210]:
porcentaje_lista(fraude.map(lambda x: (horas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), cantidad_total)

[(10, 1.0, 0.0056),
 (110, 1.0, 0.0056),
 (135, 1.0, 0.0056),
 (180, 1.0, 0.0056),
 (185, 1.0, 0.0056),
 (200, 1.0, 0.0056),
 (205, 1.0, 0.0056),
 (250, 1.0, 0.0056),
 (270, 1.0, 0.0056),
 (275, 1.0, 0.0056)]

Cantidad respecto de días

In [211]:
porcentaje_lista(sorted(fraude.map(lambda x: (dias(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect()), cantidad_total)

[(1, 112.0, 0.637),
 (2, 123.0, 0.6996),
 (3, 92.0, 0.5233),
 (4, 115.0, 0.6541),
 (5, 127.0, 0.7224),
 (6, 99.0, 0.5631),
 (7, 136.0, 0.7736),
 (8, 93.0, 0.529),
 (9, 119.0, 0.6769),
 (10, 90.0, 0.5119),
 (11, 123.0, 0.6996),
 (12, 84.0, 0.4778),
 (13, 82.0, 0.4664),
 (14, 126.0, 0.7167),
 (15, 143.0, 0.8134),
 (16, 70.0, 0.3981),
 (17, 126.0, 0.7167),
 (18, 161.0, 0.9158),
 (19, 118.0, 0.6712),
 (20, 135.0, 0.7679),
 (21, 116.0, 0.6598),
 (22, 147.0, 0.8361),
 (23, 106.0, 0.6029),
 (24, 76.0, 0.4323),
 (25, 63.0, 0.3583),
 (26, 82.0, 0.4664),
 (27, 100.0, 0.5688),
 (28, 150.0, 0.8532),
 (29, 129.0, 0.7337),
 (30, 158.0, 0.8987),
 (31, 149.0, 0.8475),
 (32, 51.0, 0.2901),
 (33, 123.0, 0.6996),
 (34, 159.0, 0.9044),
 (35, 120.0, 0.6825),
 (36, 135.0, 0.7679),
 (37, 126.0, 0.7167),
 (38, 110.0, 0.6257),
 (39, 117.0, 0.6655),
 (40, 109.0, 0.62),
 (41, 121.0, 0.6882),
 (42, 85.0, 0.4835),
 (43, 160.0, 0.9101),
 (44, 119.0, 0.6769),
 (45, 100.0, 0.5688),
 (46, 117.0, 0.6655),
 (47, 91.0, 0

Top ten cantidad respecto de días

In [212]:
porcentaje_lista(fraude.map(lambda x: (dias(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), cantidad_total)

[(64, 234.0, 1.331),
 (117, 194.0, 1.1035),
 (85, 187.0, 1.0637),
 (65, 183.0, 1.0409),
 (120, 181.0, 1.0295),
 (91, 180.0, 1.0238),
 (50, 175.0, 0.9954),
 (59, 168.0, 0.9556),
 (62, 165.0, 0.9385),
 (116, 163.0, 0.9271)]

Last ten cantidad respecto de días

In [213]:
porcentaje_lista(fraude.map(lambda x: (dias(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), cantidad_total)

[(152, 31.0, 0.1763),
 (32, 51.0, 0.2901),
 (139, 61.0, 0.3469),
 (25, 63.0, 0.3583),
 (122, 64.0, 0.364),
 (80, 66.0, 0.3754),
 (149, 66.0, 0.3754),
 (145, 67.0, 0.3811),
 (101, 68.0, 0.3868),
 (150, 69.0, 0.3924)]

Cantidad respecto de semanas

In [214]:
porcentaje_lista(sorted(fraude.map(lambda x: (semanas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect()), cantidad_total)

[(1, 804.0, 4.5733),
 (2, 717.0, 4.0784),
 (3, 869.0, 4.9431),
 (4, 724.0, 4.1183),
 (5, 889.0, 5.0568),
 (6, 803.0, 4.5676),
 (7, 823.0, 4.6814),
 (8, 862.0, 4.9032),
 (9, 927.0, 5.273),
 (10, 996.0, 5.6655),
 (11, 880.0, 5.0056),
 (12, 676.0, 3.8452),
 (13, 912.0, 5.1877),
 (14, 884.0, 5.0284),
 (15, 718.0, 4.0841),
 (16, 866.0, 4.926),
 (17, 1069.0, 6.0807),
 (18, 833.0, 4.7383),
 (19, 727.0, 4.1353),
 (20, 611.0, 3.4755),
 (21, 639.0, 3.6348),
 (22, 351.0, 1.9965)]

Top ten cantidad respecto de semanas

In [215]:
porcentaje_lista(fraude.map(lambda x: (semanas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), cantidad_total)

[(17, 1069.0, 6.0807),
 (10, 996.0, 5.6655),
 (9, 927.0, 5.273),
 (13, 912.0, 5.1877),
 (5, 889.0, 5.0568),
 (14, 884.0, 5.0284),
 (11, 880.0, 5.0056),
 (3, 869.0, 4.9431),
 (16, 866.0, 4.926),
 (8, 862.0, 4.9032)]

Last ten cantidad respecto de semanas

In [216]:
porcentaje_lista(fraude.map(lambda x: (semanas(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), cantidad_total)

[(22, 351.0, 1.9965),
 (20, 611.0, 3.4755),
 (21, 639.0, 3.6348),
 (12, 676.0, 3.8452),
 (2, 717.0, 4.0784),
 (15, 718.0, 4.0841),
 (4, 724.0, 4.1183),
 (19, 727.0, 4.1353),
 (6, 803.0, 4.5676),
 (1, 804.0, 4.5733)]

Cantidad respecto de meses

In [217]:
porcentaje_lista(sorted(fraude.map(lambda x: (meses(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).collect()), cantidad_total)

[(1, 3114.0, 17.7133),
 (2, 3377.0, 19.2093),
 (3, 3479.0, 19.7895),
 (4, 3380.0, 19.2263),
 (5, 3240.0, 18.43),
 (6, 990.0, 5.6313)]

In [218]:
porcentaje_tupla(fraude.map(lambda x: (meses(x.timestamp), 1)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), cantidad_total)

(3, 3479.0, 19.78)

Monto respecto del tiempo

Máximo monto respecto de segundos

In [220]:
monto_total = fraude.map(lambda x: (1, x.ammount)).reduceByKey(lambda a,b: a + b).collect()[0][1]
(fraude.map(lambda x: segundos(x.timestamp)).reduce(lambda a,b: a if a > b else b), redondeo(monto_total, 2), 100)

(13064705, 2614236.33, 100)

Top ten monto respecto de minutos

In [221]:
porcentaje_lista(fraude.map(lambda x: (minutos(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), monto_total)

[(176918, 6239.6, 0.2386),
 (99097, 4191.0, 0.1603),
 (199979, 3822.95, 0.1462),
 (189819, 3280.05, 0.1254),
 (87835, 3247.16, 0.1242),
 (120792, 3221.0, 0.1232),
 (151504, 3191.0, 0.122),
 (89633, 3190.0, 0.122),
 (151593, 3190.0, 0.122),
 (181769, 3190.0, 0.122)]

Last ten monto respecto de minutos

In [222]:
porcentaje_lista(fraude.map(lambda x: (minutos(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), monto_total)

[(77470, 0.29, 0.0),
 (77302, 0.29, 0.0),
 (77307, 0.29, 0.0),
 (11188, 0.29, 0.0),
 (77311, 0.35, 0.0),
 (15646, 0.42, 0.0),
 (15588, 0.42, 0.0),
 (15594, 0.42, 0.0),
 (15639, 0.42, 0.0),
 (8390, 0.46, 0.0)]

Top ten monto respecto de horas

In [223]:
porcentaje_lista(fraude.map(lambda x: (horas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), monto_total)

[(3000, 10744.97, 0.411),
 (2949, 10068.41, 0.3851),
 (3217, 9004.46, 0.3444),
 (2086, 6795.08, 0.2599),
 (2875, 6710.91, 0.2567),
 (2991, 6565.0, 0.2511),
 (2973, 6307.91, 0.2412),
 (2879, 6088.46, 0.2328),
 (3178, 6031.0, 0.2306),
 (3384, 5995.85, 0.2293)]

Last ten monto respecto de horas

In [224]:
porcentaje_lista(fraude.map(lambda x: (horas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), monto_total)

[(310, 2.44, 0.0),
 (3057, 6.16, 0.0002),
 (3297, 7.65, 0.0002),
 (2477, 8.21, 0.0003),
 (3588, 8.91, 0.0003),
 (2334, 9.54, 0.0003),
 (2433, 9.99, 0.0003),
 (2309, 10.35, 0.0003),
 (3441, 11.06, 0.0004),
 (2313, 12.68, 0.0004)]

Monto respecto de días

In [225]:
porcentaje_lista(sorted(fraude.map(lambda x: (dias(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).collect()), monto_total)

[(1, 20337.14, 0.7779),
 (2, 15359.6, 0.5875),
 (3, 14254.33, 0.5452),
 (4, 14120.44, 0.5401),
 (5, 16692.72, 0.6385),
 (6, 12869.08, 0.4922),
 (7, 21670.36, 0.8289),
 (8, 13508.15, 0.5167),
 (9, 13106.59, 0.5013),
 (10, 12709.72, 0.4861),
 (11, 21725.53, 0.831),
 (12, 14710.55, 0.5627),
 (13, 10975.38, 0.4198),
 (14, 15782.4, 0.6037),
 (15, 16438.68, 0.6288),
 (16, 7279.53, 0.2784),
 (17, 14338.32, 0.5484),
 (18, 19745.87, 0.7553),
 (19, 13647.86, 0.522),
 (20, 20877.6, 0.7986),
 (21, 11047.98, 0.4226),
 (22, 18677.98, 0.7144),
 (23, 15191.02, 0.581),
 (24, 11902.08, 0.4552),
 (25, 5965.49, 0.2281),
 (26, 12609.16, 0.4823),
 (27, 14725.04, 0.5632),
 (28, 22367.33, 0.8555),
 (29, 16953.83, 0.6485),
 (30, 22107.92, 0.8456),
 (31, 18823.93, 0.72),
 (32, 6477.42, 0.2477),
 (33, 18623.23, 0.7123),
 (34, 22765.2, 0.8708),
 (35, 15812.87, 0.6048),
 (36, 20959.42, 0.8017),
 (37, 24458.92, 0.9356),
 (38, 13548.03, 0.5182),
 (39, 15022.65, 0.5746),
 (40, 16653.4, 0.637),
 (41, 18989.12, 0.7263)

Top ten monto respecto de días

In [226]:
porcentaje_lista(fraude.map(lambda x: (dias(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), monto_total)

[(120, 39055.2, 1.4939),
 (125, 34599.42, 1.3235),
 (109, 32184.5, 1.2311),
 (95, 31478.14, 1.2041),
 (106, 30852.04, 1.1801),
 (114, 29092.98, 1.1128),
 (85, 28536.53, 1.0915),
 (64, 27517.51, 1.0526),
 (112, 27171.11, 1.0393),
 (132, 25954.06, 0.9927)]

Last ten monto respecto de días

In [227]:
porcentaje_lista(fraude.map(lambda x: (dias(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), monto_total)

[(25, 5965.49, 0.2281),
 (32, 6477.42, 0.2477),
 (16, 7279.53, 0.2784),
 (152, 7747.41, 0.2963),
 (150, 7782.21, 0.2976),
 (149, 7826.8, 0.2993),
 (128, 7897.82, 0.3021),
 (67, 8018.99, 0.3067),
 (145, 8080.86, 0.3091),
 (80, 8365.21, 0.3199)]

Monto respecto de semanas

In [228]:
porcentaje_lista(sorted(fraude.map(lambda x: (semanas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).collect()), monto_total)

[(1, 115303.69, 4.4106),
 (2, 102518.35, 3.9215),
 (3, 103375.85, 3.9543),
 (4, 101438.13, 3.8802),
 (5, 121564.43, 4.65),
 (6, 122015.66, 4.6673),
 (7, 104885.8, 4.0121),
 (8, 118201.74, 4.5214),
 (9, 127460.33, 4.8756),
 (10, 134119.43, 5.1303),
 (11, 108111.65, 4.1354),
 (12, 92534.23, 3.5396),
 (13, 147655.6, 5.6481),
 (14, 146352.54, 5.5982),
 (15, 123198.73, 4.7126),
 (16, 167064.38, 6.3905),
 (17, 158979.25, 6.0812),
 (18, 157838.82, 6.0376),
 (19, 114772.7, 4.3902),
 (20, 105325.59, 4.0289),
 (21, 94819.16, 3.627),
 (22, 46700.17, 1.7863)]

Top ten monto respecto de semanas

In [229]:
porcentaje_lista(fraude.map(lambda x: (semanas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: -x[1]), monto_total)

[(16, 167064.38, 6.3905),
 (17, 158979.25, 6.0812),
 (18, 157838.82, 6.0376),
 (13, 147655.6, 5.6481),
 (14, 146352.54, 5.5982),
 (10, 134119.43, 5.1303),
 (9, 127460.33, 4.8756),
 (15, 123198.73, 4.7126),
 (6, 122015.66, 4.6673),
 (5, 121564.43, 4.65)]

Last ten monto respecto de semanas

In [230]:
porcentaje_lista(fraude.map(lambda x: (semanas(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).takeOrdered(10, lambda x: x[1]), monto_total)

[(22, 46700.17, 1.7863),
 (12, 92534.23, 3.5396),
 (21, 94819.16, 3.627),
 (4, 101438.13, 3.8802),
 (2, 102518.35, 3.9215),
 (3, 103375.85, 3.9543),
 (7, 104885.8, 4.0121),
 (20, 105325.59, 4.0289),
 (11, 108111.65, 4.1354),
 (19, 114772.7, 4.3902)]

Monto respecto de meses

In [231]:
porcentaje_lista(sorted(fraude.map(lambda x: (meses(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).collect()), monto_total)

[(1, 422636.04, 16.1667),
 (2, 466667.65, 17.851),
 (3, 462225.65, 17.681),
 (4, 584271.26, 22.3495),
 (5, 536916.38, 20.5381),
 (6, 141519.33, 5.4134)]

Máximo monto respecto de meses

In [232]:
porcentaje_tupla(fraude.map(lambda x: (meses(x.timestamp), x.ammount)).reduceByKey(lambda a,b: a + b).reduce(lambda a,b: a if a[1] > b[1] else b), monto_total)

(4, 584271.26, 22.34)

Plataformas fraudulentas

In [ ]:
identity_rdd = sqlContext.read.csv('identity_features_train.csv', header=True, inferSchema=True).rdd

In [ ]:
def unicidad_sistema(string) -> str:
    if string == None:
      return "OTRO"
    lista = ["WINDOWS", "MOTO", "SAMSUNG", "LG", "SM", "GT", "RV", "ZTE", "HUAMWI", "IOS DEVICE","MACOS", "NOKIA", "ALCATEL", "IPHONE" ,"BUILD"]
    for sistema in lista:
        match string.upper().find(sistema):
            case -1:
                continue
            case _:
                return sistema
    return "OTRO"

def unicidad_modelo(string) -> str:
    if string == None:
      return "OTRO"
    lista = ["CHROME", "SAFARI", "FIREFOX", "SAMSUNG", "OPERA", "EDGE", "ANDROID"]
    for modelo in lista:
        match string.upper().find(modelo):
            case -1:
                continue
            case _:
                return modelo
    return "OTRO"

def nulo(numero):
  if numero == None:
    return 0
  else:
    return numero

In [ ]:
identity_rdd.first()

Row(_c0=0, Id=2987004, feature01=0.0, feature02=70787.0, feature03=None, feature04=None, feature05=None, feature06=None, feature07=None, feature08=None, feature09=None, feature10=None, feature11=100.0, feature12='NotFound', feature13=None, feature14=-480.0, feature15='New', feature16='NotFound', feature17=166.0, feature18=None, feature19=542.0, feature20=144.0, feature21=None, feature22=None, feature23=None, feature24=None, feature25=None, feature26=None, feature27=None, feature28='New', feature29='NotFound', feature30='Android 7.0', feature31='samsung browser 6.2', feature32=32.0, feature33='2220x1080', feature34='match_status:2', feature35='T', feature36='F', feature37='T', feature38='T', Device='mobile', DeviceMetadata='SAMSUNG SM-G892A Build/NRD90M')

In [ ]:
plataformas = identity_rdd.map(lambda x: (x.Id, (x.Device, x.DeviceMetadata, x.feature31))).leftOuterJoin(transactions_rdd.map(lambda x: (x.Id, x.target)))
plataformas_fraudulentas = plataformas.filter(lambda x: x[1][1] == 1).cache()
plataformas_legales = plataformas.filter(lambda x: x[1][1] == 0).cache()

In [ ]:
plataformas.first()

(2987040, (('desktop', 'Windows', 'chrome 62.0'), 0))

Fraude por dispositivo

In [ ]:
dispositivos = plataformas_fraudulentas.map(lambda x: (x[1][0][0], 1)).reduceByKey(lambda a,b: a+b).leftOuterJoin(plataformas_legales\
.map(lambda x: (x[1][0][0], 1)).reduceByKey(lambda a,b: a+b)).map(lambda x: (x[0], x[1][0], x[1][0] * 100 / (x[1][0] + x[1][1])))
dispositivos.collect()

[(None, 71, 2.417432754511406),
 ('mobile', 4726, 9.861447291544946),
 ('desktop', 4758, 6.369136860141358)]

Máximo dispositivo respecto de la cantidad

In [ ]:
dispositivos.reduce(lambda a,b: a if a[1] > b[1] else b)

('desktop', 4758, 6.369136860141358)

Máximo dispositivo respecto del promedio

In [ ]:
dispositivos.reduce(lambda a,b: a if a[2] > b[2] else b)

('mobile', 4726, 9.861447291544946)

Fraude por sistema

In [ ]:
sistema = plataformas_fraudulentas.map(lambda x: (unicidad_sistema(x[1][0][1]), 1)).reduceByKey(lambda a,b: a+b)\
.leftOuterJoin(plataformas_legales.map(lambda x: (unicidad_sistema(x[1][0][1]), 1)).reduceByKey(lambda a,b: a+b))\
.map(lambda x: (x[0], x[1][0], x[1][0] * 100 / (x[1][0] + nulo(x[1][1]))))
sistema.collect()

[('BUILD', 1035, 16.512444160816848),
 ('MOTO', 377, 15.721434528773978),
 ('MACOS', 218, 1.9339957416607523),
 ('RV', 297, 7.4811083123425695),
 ('GT', 20, 15.037593984962406),
 ('OTRO', 2484, 8.279172082791721),
 ('WINDOWS', 2610, 6.309834638816362),
 ('ZTE', 9, 20.0),
 ('IOS DEVICE', 1079, 6.113314447592068),
 ('SM', 984, 12.479391249207355),
 ('NOKIA', 10, 100.0),
 ('SAMSUNG', 182, 7.882200086617583),
 ('ALCATEL', 7, 13.20754716981132),
 ('LG', 242, 10.980036297640654),
 ('IPHONE', 1, 100.0)]

Top five sistema respecto de la cantidad

In [ ]:
sistema.takeOrdered(5, lambda x: -x[1])

('WINDOWS', 2610, 6.309834638816362)

Top five sistema respecto del promedio

In [ ]:
sistema.takeOrdered(5, lambda x: -x[2])

('IPHONE', 1, 100.0)

Fraude por modelo

In [ ]:
modelo = plataformas_fraudulentas.map(lambda x: (unicidad_modelo(x[1][0][2]), 1)).reduceByKey(lambda a,b: a+b)\
.leftOuterJoin(plataformas_legales.map(lambda x: (unicidad_modelo(x[1][0][2]), 1)).reduceByKey(lambda a,b: a+b))\
.map(lambda x: (x[0], x[1][0], x[1][0] * 100 / (x[1][0] + nulo(x[1][1]))))
modelo.collect()

[('ANDROID', 92, 20.444444444444443),
 ('CHROME', 6158, 9.296637932335935),
 ('OTRO', 319, 2.5892857142857144),
 ('SAFARI', 2052, 6.18016444297202),
 ('EDGE', 177, 3.68673193084774),
 ('FIREFOX', 470, 7.4603174603174605),
 ('SAMSUNG', 158, 8.24634655532359),
 ('OPERA', 129, 38.392857142857146)]

Top five modelo respecto de la cantidad

In [ ]:
modelo.takeOrdered(5, lambda x: -x[1])

('CHROME', 6158, 9.296637932335935)

Top five modelo respecto del promedio

In [ ]:
modelo.takeOrdered(5, lambda x: -x[2])

('OPERA', 129, 38.392857142857146)

Fraudes de tarjetas

In [ ]:
tarjetas = fraude.map(lambda x: (x.feat_card_4, 1)).reduceByKey(lambda a,b: a + b)\
.leftOuterJoin(transactions_rdd.map(lambda x: (x.feat_card_4, 1)).reduceByKey(lambda a,b: a + b))\
.map(lambda x: (x[0], x[1][0], x[1][0] * 100 / (x[1][0] + x[1][1])))
tarjetas.collect()

[(None, 26, 3.026775320139697),
 ('visa', 11327, 3.33980044345898),
 ('american express', 218, 2.827496757457847),
 ('mastercard', 5574, 3.3647838606276825),
 ('discover', 435, 7.060542119785749)]

Top three cantidad de tarjeta

In [ ]:
tarjetas.takeOrdered(5, lambda x: -x[1])

('visa', 11327, 3.33980044345898)

Top three promedio de tarjeta

In [ ]:
tarjetas.takeOrdered(5, lambda x: -x[2])

('discover', 435, 7.060542119785749)

analizar parte legal

Fraude por tipo de pago

In [ ]:
banco = fraude.map(lambda x: (x.feat_card_6, 1)).reduceByKey(lambda a,b: a + b)\
.leftOuterJoin(transactions_rdd.map(lambda x: (x.feat_card_6, 1)).reduceByKey(lambda a,b: a + b))\
.map(lambda x: (x[0], x[1][0], x[1][0] * 100 / (x[1][0] + x[1][1])))
banco.collect()

[('credit', 8626, 6.304311283591689),
 (None, 24, 2.816901408450704),
 ('debit', 8930, 2.3388290140513077)]

Máxima cantidad de tipo de pago

In [ ]:
banco.reduce(lambda a,b: a if a[1] > b[1] else b)

('debit', 8930, 2.3388290140513077)

Máximo promodio por tipo de pago

In [ ]:
banco.reduce(lambda a,b: a if a[2] > b[2] else b)

('credit', 8626, 6.304311283591689)

Fraude segun el correo

In [ ]:
def unicidad_correo(string) -> str:
  if string == None:
    return "otro"
  lista = ["yahoo", "gmail", "hotmail", "outlook", "live"]
  for correo in lista:
    match string.lower().find(correo):
      case -1:
        continue
      case _:
        return f"{correo}.com"
  return string

In [ ]:
correo = fraude.map(lambda x: (x.payer_email_domain, 1)).reduceByKey(lambda a,b: a + b)\
.leftOuterJoin(transactions_rdd.map(lambda x: (x.payer_email_domain, 1)).reduceByKey(lambda a,b: a + b))\
.map(lambda x: (x[0], x[1][0], x[1][0] * 100 / (x[1][0] + x[1][1])))
correo.collect()

[(None, 2346, 2.9044718466795425),
 ('comcast.net', 203, 2.866017224339969),
 ('msn.com', 62, 1.7371812832726254),
 ('earthlink.net', 6, 1.348314606741573),
 ('yahoo.fr', 5, 4.504504504504505),
 ('gmail.com', 8503, 4.194414025118142),
 ('verizon.net', 15, 0.6265664160401002),
 ('bellsouth.net', 47, 2.712060011540681),
 ('outlook.es', 56, 12.389380530973451),
 ('suddenlink.net', 4, 3.1746031746031744),
 ('rocketmail.com', 2, 0.3552397868561279),
 ('embarqmail.com', 7, 2.9914529914529915),
 ('yahoo.es', 2, 1.680672268907563),
 ('mac.com', 11, 2.842377260981912),
 ('hotmail.com', 2022, 4.9124170938509755),
 ('frontiernet.net', 5, 2.9761904761904763),
 ('mail.com', 96, 16.63778162911612),
 ('charter.net', 16, 2.1828103683492497),
 ('gmail', 8, 1.8475750577367205),
 ('protonmail.com', 31, 31.31313131313131),
 ('yahoo.com.mx', 15, 1.1061946902654867),
 ('optonline.net', 17, 1.89520624303233),
 ('juno.com', 6, 2.04778156996587),
 ('outlook.com', 390, 8.304940374787053),
 ('anonymous.com', 731

Top ten cantidad de correo

In [ ]:
correo.takeOrdered(10, lambda x: -x[1])

('gmail.com', 8503, 4.194414025118142)

Top ten promedio de correo

In [ ]:
correo.takeOrdered(5, lambda x: -x[2])

('protonmail.com', 31, 31.31313131313131)